In [1]:
import pandas as pd

df = pd.read_csv(r'..\Data\Clean\carchive_M6.csv')

df2 = df[df.acq_date == '2018-10-31'].copy()

display(df2.head())

,latitude,longitude,brightness,scan,track,acq_date,confidence,bright_t31,frp
4697436,-12.9495,134.9012,332.3,1.3,1.1,2018-10-31,72,302.2,19.4
4697437,-12.9477,134.8896,331.4,1.3,1.1,2018-10-31,70,304.4,17.9
4697438,-12.2995,134.8708,340.6,1.3,1.1,2018-10-31,86,303.4,38.4
4697439,-12.9867,134.1184,344.4,1.4,1.2,2018-10-31,89,308.2,58.2
4697440,-12.9888,134.1314,333.6,1.4,1.2,2018-10-31,75,306.3,28.8


In [2]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure, ColumnDataSource
from bokeh.tile_providers import get_provider, Vendors
from bokeh.palettes import PRGn, RdYlGn
from bokeh.transform import linear_cmap,factor_cmap
from bokeh.layouts import row, column
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter

import numpy as np


# Define function to switch from lat/long to mercator coordinates
def x_coord(x, y):
    
    lat = x
    lon = y
    
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)

# Define coord as tuple (lat,long)
df2['coordinates'] = list(zip(df2['latitude'], df2['longitude']))


# Obtain list of mercator coordinates
mercators = [x_coord(x, y) for x, y in df2['coordinates'] ]


In [6]:
import folium
import folium.plugins as plugins
f = folium.Figure(width=1000, height=1000)
center_lat = -24.003249 
center_long = 133.737310
m = folium.Map(location=[center_lat,center_long], control_scale=True, zoom_start=4,width=750, height=500,zoom_control=True).add_to(f)
#for i in range(0,df2.shape[0]):    
#    location=[df2.iloc[i]['latitude'], df2.iloc[i]['longitude']]
#    folium.CircleMarker(location,radius=1,color='red').add_to(m)
data_heat = df2[['latitude','longitude','frp']].values.tolist()
plugins.HeatMap(data_heat).add_to(m)
display(m)

In [29]:

# Create mercator column in our df
df2['mercator'] = mercators

# Split that column out into two separate cols - mercator_x and mercator_y
df2[['mercator_x', 'mercator_y']] = df2['mercator'].apply(pd.Series)

# Drop 'geometry' column 
#df = df.drop(columns=['daynight', 'satellite', 'acq_time', 'version'])

# Examine our modified DataFrame
df2.head()



,latitude,longitude,brightness,scan,track,acq_date,confidence,bright_t31,frp,coordinates,mercator,mercator_x,mercator_y
4697436,-12.9495,134.9012,332.3,1.3,1.1,2018-10-31,72,302.2,19.4,"(-12.9495, 134.9012)","(15017132.891401555, -1453963.351699395)",1.501713e+07,-1.453963e+06
4697437,-12.9477,134.8896,331.4,1.3,1.1,2018-10-31,70,304.4,17.9,"(-12.9477, 134.8896)","(15015841.585308354, -1453757.7484282635)",1.501584e+07,-1.453758e+06
4697438,-12.2995,134.8708,340.6,1.3,1.1,2018-10-31,86,303.4,38.4,"(-12.2995, 134.8708)","(15013748.778881442, -1379812.5407460371)",1.501375e+07,-1.379813e+06
4697439,-12.9867,134.1184,344.4,1.4,1.2,2018-10-31,89,308.2,58.2,"(-12.9867, 134.1184)","(14929991.994008584, -1458212.8188270847)",1.492999e+07,-1.458213e+06
4697440,-12.9888,134.1314,333.6,1.4,1.2,2018-10-31,75,306.3,28.8,"(-12.9888, 134.1314)","(14931439.147388896, -1458452.7270570712)",1.493144e+07,-1.458453e+06


In [26]:
import reverse_geocoder as rg
coords = list(zip(df2['latitude'], df2['longitude']))
print(rg.search(coords))

Loading formatted geocoded file...
[OrderedDict([('lat', '-13.85413'), ('lon', '136.42129'), ('name', 'Alyangula'), ('admin1', 'Northern Territory'), ('admin2', ''), ('cc', 'AU')]), OrderedDict([('lat', '-13.85413'), ('lon', '136.42129'), ('name', 'Alyangula'), ('admin1', 'Northern Territory'), ('admin2', ''), ('cc', 'AU')]), OrderedDict([('lat', '-12.18652'), ('lon', '136.78201'), ('name', 'Nhulunbuy'), ('admin1', 'Northern Territory'), ('admin2', 'East Arnhem'), ('cc', 'AU')]), OrderedDict([('lat', '-14.46517'), ('lon', '132.26347'), ('name', 'Katherine'), ('admin1', 'Northern Territory'), ('admin2', 'Katherine'), ('cc', 'AU')]), OrderedDict([('lat', '-14.46517'), ('lon', '132.26347'), ('name', 'Katherine'), ('admin1', 'Northern Territory'), ('admin2', 'Katherine'), ('cc', 'AU')]), OrderedDict([('lat', '-12.55397'), ('lon', '131.11165'), ('name', 'McMinns Lagoon'), ('admin1', 'Northern Territory'), ('admin2', 'Litchfield'), ('cc', 'AU')]), OrderedDict([('lat', '-13.85413'), ('lon', '

In [ ]:
df2['acq_date'] = pd.to_datetime(df2['acq_date']) - pd.to_timedelta(7, unit='d')
def wmy(df):
    return ([g for n, g in df.set_index('acq_date').groupby(pd.Grouper(freq = 'W'))],
            [g for n, g in df.set_index('acq_date').groupby(pd.Grouper(freq = 'M'))],
            [g for n, g in df.set_index('acq_date').groupby(pd.Grouper(freq = 'Y'))])
weeks, months, years = wmy(df2)



In [ ]:
# Create map with Bokeh

# Select tile set to use
chosentile = get_provider(Vendors.STAMEN_TONER)

# Choose palette
palette = PRGn[11]

# Tell Bokeh to use df as the source of the data
source = ColumnDataSource(data=df2)

# Define color mapper - which column will define the colour of the data points
color_mapper = linear_cmap(field_name = 'brightness', palette = palette, low = df2['brightness'].min(), high = df2['brightness'].max())

# Set tooltips - these appear when we hover over a data point in our map, very nifty and very useful
nan_color = '#d9d9d9'
tooltips = [("Brightness","@brightness"), ("frp","@frp")]


# Create figure
p = figure(title = 'Australia fires', x_axis_type="mercator", y_axis_type="mercator", 
           x_axis_label = 'Longitude', y_axis_label = 'Latitude', tooltips = tooltips)

# Add map tile
p.add_tile(chosentile)

# Add points using mercator coordinates
p.circle(x = 'mercator_x', y = 'mercator_y', color = color_mapper, source=source, size=30, fill_alpha = 0.7)

#Defines color bar
color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     formatter = NumeralTickFormatter(format='0.0[0000]'), 
                     label_standoff = 13, width=8, location=(0,0))

# Set color_bar location
p.add_layout(color_bar, 'right')

# Display in notebook
output_notebook()

# Save as HTML
output_file('australia.html', title='australia fires')

# Show map
show(p)



Loading BokehJS ...